# 뉴스 데이터 전처리

이 노트북은 크롤링한 뉴스 데이터를 BERT 모델 학습을 위해 전처리하는 과정을 담고 있습니다.

In [12]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

## 1. 데이터 로드 및 병합

In [2]:
# 데이터 경로 설정
data_path = Path('../data')

# 모든 CSV 파일 리스트 가져오기
csv_files = list(data_path.glob('*.csv'))

# 데이터프레임 리스트 생성
dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    # 언론사 이름 추출 (파일명에서)
    press_name = file.stem.split('_')[0]
    df['press'] = press_name
    dfs.append(df)

# 모든 데이터프레임 병합
combined_df = pd.concat(dfs, ignore_index=True)
print(f"총 {len(combined_df)} 개의 기사가 있습니다.")

총 85698 개의 기사가 있습니다.


## 2. 텍스트 전처리

In [13]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    
    # URL 제거
    text = re.sub(r'https?://\S+', '', text)
    
    # HTML 태그 제거
    text = re.sub(r'<[^>]+>', '', text)
    
    # 줄바꿈/개행 문자를 공백으로 변환
    text = re.sub(r'[\n\r]+', ' ', text)
    
    # 특수문자 제거 (한글, 영문자, 숫자만 남김)
    text = re.sub(r'[^\w\s가-힣]', ' ', text)
    
    # 연속된 공백 제거
    text = re.sub(r'\s+', ' ', text)
    
    # 문장 시작과 끝의 공백 제거
    text = text.strip()
    
    return text

In [4]:
# 결측치 제거
combined_df = combined_df.dropna(subset=['title', 'content'])

# 제목과 본문 전처리
combined_df['title_cleaned'] = combined_df['title'].apply(clean_text)
combined_df['content_cleaned'] = combined_df['content'].apply(clean_text)

# 제목과 본문 결합
combined_df['text'] = combined_df['title_cleaned'] + ' ' + combined_df['content_cleaned']

# 너무 짧은 기사 제거 (예: 100자 미만)
combined_df = combined_df[combined_df['text'].str.len() >= 100]

# 중복 기사 제거 (제목과 본문이 동일한 경우)
combined_df = combined_df.drop_duplicates(subset=['title_cleaned', 'content_cleaned'])

# 중복 제거 후 데이터 수 확인
print(f"중복 제거 후 남은 기사 수: {len(combined_df)}")

중복 제거 후 남은 기사 수: 79023


In [8]:
# 전처리된 통합 데이터셋 저장
combined_df[["title_cleaned", "content_cleaned", "text", "press", "created_date"]].to_csv("../data/전체통합.csv", index=False)

## 3. 레이블링

언론사별로 편향성 레이블을 부여합니다.

In [ ]:
# 언론사별 편향성 레이블 매핑
